# **1. Przedstawienie założeń projektu końcowego**

# Projekt zakłada stworzenie modelu klasyfikacyjnego, który na podstawie zgromadzonych wcześniej danych (tekstów piosenek ze strony wywrota.pl oraz gatunków muzycznych wykonawców ze strony wikipedia.pl) po przekształceniach i wytrenowaniu modelu będzie potrafił klasyfikować gatunki muzyczne piosenek na podstawie dostarczonego tekstu. Projekt zakłada, że piosenki będą w języku polskim, ograniczone początkowo do polskich wykonawców (z możliwością dodania przetłumaczonych tekstów z innych języków na polski - w dalszym rozwoju projektu). Dodatkowym działaniem będzie zastosowanie miary sentymentu poszczególnych słów w języku polskim przy użyciu biblioteki TextBlob oraz dodanie ich do modelu danych jako dodatkowa informacja dla modelu

1. Pobranie potrzebnych bibliotek

In [1]:
!pip install user_agent

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for user_agent: filename=user_agent-0.1.10-py3-none-any.whl size=18982 sha256=ab4ff776ea47bc4664935fcffd2e601a9ce14a0f9fe4fb0e9c088ad257b52c5c
  Stored in directory: c:\users\kamil\appdata\local\pip\cache\wheels\69\29\26\1956a891a058037774285ee79ab5c3ecf034dba50a4198fedd
Successfully built user_agent


In [28]:
!pip install xlsxwriter

     -------------------------------------- 150.0/150.0 kB 1.1 MB/s eta 0:00:00


In [29]:
import requests
from bs4 import BeautifulSoup
import lxml.html
from user_agent import generate_user_agent
import numpy as np
import pandas as pd
import xlsxwriter

2. Stowrzenie połączenia ze stroną wywrota.pl, stoujemy user_agenta żeby nie dostać bana za przeciążanie sieci (dużo requestów do bazy), pobieramy kod html strony i szukamy potrzebnych danych (przez inspekcję strony :), nie w kodzie na szczęście), printujemy kod żeby zobaczyć drzewo html

In [3]:
url = 'https://spiewnik.wywrota.pl/country/PL/artists'
user_agent = generate_user_agent(os=('mac', 'linux', 'win'))
r = requests.get(url, headers={'user_agent': user_agent})
soup = BeautifulSoup(r.text, "html.parser")
soup

<!DOCTYPE html>

<html lang="pl">
<head>
<meta charset="utf-8"/>
<meta content="index,follow" name="robots"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<meta content="t3VQAIiyBWKktU4OhIpQw1LCfyglBJBq8DxrLlzq" name="csrf-token"/>
<title>Polscy wykonawcy alfabetycznie</title>
<meta content="Śpiewnik Wywroty" property="og:site_name">
<meta content="Polscy wykonawcy alfabetycznie" property="og:title">
<meta content="article" property="og:type">
<meta content="https://static.wywrota.pl/gfx/20lat/2018-spiewnik-logo.jpg" property="og:image">
<meta content="summary" name="twitter:card"/>
<meta content="@WywrotaPL" name="twitter:site"/>
<meta content="Polscy wykonawcy alfabetycznie" name="twitter:title"/>
<script type="application/ld+json">{"@context":"https://schema.org","@type":"WebPage","name":"Polscy wykonawcy alfabetycznie","description":"For those who helped create the Genki Dama"}</script>
<link href="https://spiewnik.wywrota.pl/assets/img/fa

3. Widać że linki do podstron są zapisane w znacznikach <%a> w parametrze href więc szukamy wszystkich znaczników <%a> które zawierają parametr href oraz szukamy wykonawców w znaczniku <%h4> i konwertujemy nasz ResultSet do listy

In [4]:
body = soup.body
titles_clean = [title.string for title in soup.find_all('h4')]
len(titles_clean)
print(f'Długość listy: {len(titles_clean)} oraz lista {titles_clean}')

Długość listy: 314 oraz lista ['Akcent', 'Anna Jantar', 'Arka Noego', 'Andrzej Korycki', 'Agnieszka Chylińska', 'Ania Wyszkoni', 'Andrzej Rosiewicz', 'Akurat', 'Artur Andrus', 'Andrzej Rybiński', 'Andrzej Zaucha', 'Ania Dąbrowska', 'Budka Suflera', 'Bajm', 'Big Cyc', 'Baciary', 'Boys', 'Bayer Full', 'Breakout', 'Baciary', 'Brathanki', 'Bracia Figo Fagot', 'Barbie i Diamentowy Pałac', 'Bohdan Smoleń', 'Czerwone Gitary', 'Czesław Niemen', 'Chłopcy z Placu Broni', 'classic', 'Coma', 'Cztery Refy', 'Cisza jak ta', 'Czerwony Tulipan', 'Cree', 'Czesław Śpiewa', 'Cezary Makiewicz', 'Czadoman', 'Dżem', 'Dawid Podsiadło', 'Dwa Plus Jeden', 'Domowe Melodie', 'Dom o Zielonych Progach', 'Doda', 'Dawid Kwiatkowski', 'De Mono', 'Dystans', 'Defis', 'Deus Meus', 'Dee Facto', 'Elektryczne Gitary', 'Exodus 15', 'Enej', 'Edyta Górniak', 'Edmund Fetting', 'EKT Gdynia', 'Edward Stachura', 'Edyta Bartosiewicz', 'Edyta Geppert', 'Eleni', 'Ewa Farna', 'Eugeniusz Bodo', 'Fasolki', 'Farben Lehre', 'Feel', 'Farb

4. Tworzymy listę nowych url do przechodzenia na podstrony. Będziemy musieli stworzyć 2 takie listy. 1 do przechodzenia na wykonawcę 2 do przechodzenia na piosenki wykonawcy i pobieranie tekstów

In [5]:
href_clean_1 = [link.get('href') for link in soup.find_all('a', href=True)]
href_clean_1 = href_clean_1[14:379]
href_clean_1 = [hre for hre in href_clean_1 if "PL" not in hre]
print(f'Długość listy: {len(href_clean_1)} oraz lista {href_clean_1}')

Długość listy: 314 oraz lista ['https://spiewnik.wywrota.pl/akcent', 'https://spiewnik.wywrota.pl/anna-jantar', 'https://spiewnik.wywrota.pl/arka-noego', 'https://spiewnik.wywrota.pl/andrzej-korycki', 'https://spiewnik.wywrota.pl/agnieszka-chylinska', 'https://spiewnik.wywrota.pl/ania-wyszkoni', 'https://spiewnik.wywrota.pl/andrzej-rosiewicz', 'https://spiewnik.wywrota.pl/akurat', 'https://spiewnik.wywrota.pl/artur-andrus', 'https://spiewnik.wywrota.pl/andrzej-rybinski', 'https://spiewnik.wywrota.pl/andrzej-zaucha', 'https://spiewnik.wywrota.pl/ania-dabrowska', 'https://spiewnik.wywrota.pl/budka-suflera', 'https://spiewnik.wywrota.pl/bajm', 'https://spiewnik.wywrota.pl/big-cyc', 'https://spiewnik.wywrota.pl/baciary', 'https://spiewnik.wywrota.pl/boys', 'https://spiewnik.wywrota.pl/bayer-full', 'https://spiewnik.wywrota.pl/breakout', 'https://spiewnik.wywrota.pl/baciary-1', 'https://spiewnik.wywrota.pl/brathanki', 'https://spiewnik.wywrota.pl/bracia-figo-fagot', 'https://spiewnik.wywrot

In [6]:
arajka = np.array(list(zip(titles_clean, href_clean_1)))

5. Tworzymy funkcję do przetwarzania linków z href, którą wykorzystamy zaraz w pętli for aby uzyskać wszystkie linki z href do piosenek

In [7]:
def get_all_href(url):
  user_agent = generate_user_agent(os=('mac', 'linux', 'win'))
  r = requests.get(url, headers={'user_agent': user_agent})
  soup = BeautifulSoup(r.text, "html.parser")
  href_clean_1 = [link.get('href') for link in soup.find_all('a', href=True)]
  return href_clean_1[13:-32]

In [8]:
all_hrefs = [get_all_href(hres) for hres in href_clean_1]

6. To wszystkie linki do tekstów piosenek ze strony

In [9]:
all_hrefs

[['https://spiewnik.wywrota.pl/akcent/a-tyle-moglo-byc',
  'https://spiewnik.wywrota.pl/akcent/babylon',
  'https://spiewnik.wywrota.pl/akcent/biore-urlop-od-ciebie',
  'https://spiewnik.wywrota.pl/akcent/boracay-feat-sandra-n',
  'https://spiewnik.wywrota.pl/akcent/buchet-de-trandafiri',
  'https://spiewnik.wywrota.pl/akcent/chimie-intre-noi',
  'https://spiewnik.wywrota.pl/akcent/chlopak-z-gitara',
  'https://spiewnik.wywrota.pl/akcent/czekam-na-ciebie',
  'https://spiewnik.wywrota.pl/akcent/czemu-jestes-taka-dziewczyno',
  'https://spiewnik.wywrota.pl/akcent/da-mi-dragostea',
  'https://spiewnik.wywrota.pl/akcent/delight',
  'https://spiewnik.wywrota.pl/akcent/departe-de-tine',
  'https://spiewnik.wywrota.pl/akcent/dragoste-de-inchiriat',
  'https://spiewnik.wywrota.pl/akcent/durere-fara-sfarsit',
  'https://spiewnik.wywrota.pl/akcent/enigma',
  'https://spiewnik.wywrota.pl/akcent/every-time-turn-my-life-around-feat-ronnie-bassroller',
  'https://spiewnik.wywrota.pl/akcent/faina-fea

7. Sprawdzam ile dokładnie będę miał tekstów. Jest to przybliżona ilość, ponieważ zauważyłęm że wśród linków występują inne nie odnoszące się do tekstów, które trzeba będzie wyrzucić przy przygotowywaniu danych

In [10]:
a=0
for i in all_hrefs:
  a += len(i)
print(a)

12388


8. Przygotowuje funkcję do wyodrębiebia tekstu ze stron

In [11]:
def get_all_lyrics(url):
  user_agent = generate_user_agent(os=('mac', 'linux', 'win'))
  r = requests.get(url, headers={'user_agent': user_agent})
  soup = BeautifulSoup(r.text, "html.parser")
  a = soup.find('div','interpretation-content')
  lyrics_clean =[]
  if a != None:
    for lyric in a:
      if lyric.string != '\n' and lyric.string != None:
        lyrics_clean.append(lyric.string)
  return lyrics_clean

get_all_lyrics('https://spiewnik.wywrota.pl/budka-suflera/a-po-nocy-przychodzi-dzien')

['Jest taki samotny dom',
 '\n Uderzył deszcz, wybuchła noc,',
 '\n Przy drodze pusty dwór,',
 '\n W katedrach drzew, w przyłbicach gór,',
 '\n Wagnerowski ton.',
 '\n Za witraża dziwnym szkłem,',
 '\n Pustych komnat chłód,',
 '\n W szary pył rozbity czas,',
 '\n Martwy, pusty dwór.',
 '\n Dorzucam drew, bo ogień zgasł,',
 '\n Ciągle burza trwa,',
 '\n Nagle feria barw i mnóstwo świec,',
 '\n Ktoś na skrzypcach gra,',
 '\n Gotyckie odrzwia chylą się',
 '\n I skrzypiąc suną w bok',
 '\n I biała pani płynie z nich',
 '\n W brylantowej mgle.',
 '\n Zawirował z nami dwór,',
 '\n Rudych włosów płomień,',
 '\n Nad górami lecę, lecę z nią,',
 '\n Różę trzyma w dłoni.',
 '\n A po nocy przychodzi dzień,',
 '\n A po burzy spokój,',
 '\n Nagle ptaki budzą mnie',
 '\n Tłukąc się do okien',
 '\n A po nocy przychodzi dzień,',
 '\n A po burzy spokój,',
 '\n Nagle ptaki budzą mnie',
 '\n Tłukąc się do okien',
 '\n Znowu szary, pusty dom,',
 '\n Gdzie schroniłem się',
 '\n I najmilsza z wszystkich, z w

In [12]:
get_all_lyrics('https://spiewnik.wywrota.pl/favorites/type/15/id/3141')

[]

In [21]:
flat_list_of_all_hrefs = [item for sublist in all_hrefs for item in sublist if item is not None]
flat_list_of_all_hrefs
for lis in flat_list_of_all_hrefs:
    if lis == '':
        flat_list_of_all_hrefs.remove(lis)
flat_list_of_all_hrefs

['https://spiewnik.wywrota.pl/akcent/a-tyle-moglo-byc',
 'https://spiewnik.wywrota.pl/akcent/babylon',
 'https://spiewnik.wywrota.pl/akcent/biore-urlop-od-ciebie',
 'https://spiewnik.wywrota.pl/akcent/boracay-feat-sandra-n',
 'https://spiewnik.wywrota.pl/akcent/buchet-de-trandafiri',
 'https://spiewnik.wywrota.pl/akcent/chimie-intre-noi',
 'https://spiewnik.wywrota.pl/akcent/chlopak-z-gitara',
 'https://spiewnik.wywrota.pl/akcent/czekam-na-ciebie',
 'https://spiewnik.wywrota.pl/akcent/czemu-jestes-taka-dziewczyno',
 'https://spiewnik.wywrota.pl/akcent/da-mi-dragostea',
 'https://spiewnik.wywrota.pl/akcent/delight',
 'https://spiewnik.wywrota.pl/akcent/departe-de-tine',
 'https://spiewnik.wywrota.pl/akcent/dragoste-de-inchiriat',
 'https://spiewnik.wywrota.pl/akcent/durere-fara-sfarsit',
 'https://spiewnik.wywrota.pl/akcent/enigma',
 'https://spiewnik.wywrota.pl/akcent/every-time-turn-my-life-around-feat-ronnie-bassroller',
 'https://spiewnik.wywrota.pl/akcent/faina-feat-liv',
 'https:/

In [22]:
#all_lyrics = [get_all_lyrics(hr) for hres in all_hrefs if hres is not type(None) for hr in hres if hr is not type(None)]
all_lyrics = [get_all_lyrics(hr) for hr in flat_list_of_all_hrefs if hr is not type(None)]

In [23]:
all_lyrics

[['Wiatr ślady zawieje',
  '\nNa dobre lub złe',
  '\nJesienią pójdę złotą',
  '\nZapomnę albo nie',
  '\nWiatr ślady pomiesza',
  '\nCo wczoraj, a co dziś',
  '\nNie będzie nic za nami',
  '\nA tyle mogło być',
  '\n[2x:]',
  '\nSrebrem rzeka lśni',
  '\nNiebem szumi las',
  '\nAle tam gdzie ty',
  '\nTam już nie ma nas',
  '\nGłośny ptaków śpiew',
  '\nNiebo dobrze zna',
  '\nAle tam gdzie ty',
  '\nTam już nie ma nas',
  '\nAle tam gdzie ty',
  '\nTam już nie ma nas',
  '\nŚnieg jutro zasypie',
  '\nNa biało cały świat',
  '\nOdjadę stąd daleko',
  '\nA ze mną polny wiatr',
  '\nMasz wszystko zapomnieć',
  '\nCo wczoraj, a co dziś',
  '\nNie będzie nic za nami',
  '\nA tyle mogło być',
  '\n[2x:]',
  '\nSrebrem rzeka lśni',
  '\nNiebem szumi las',
  '\nAle tam gdzie ty',
  '\nTam już nie ma nas',
  '\nGłośny ptaków śpiew',
  '\nNiebo dobrze zna',
  '\nAle tam gdzie ty',
  '\nTam już nie ma nas',
  '\nAle tam gdzie ty',
  '\nTam już nie ma nas',
  '\n[2x:]',
  '\nSrebrem rzeka lśni',

In [30]:
df = pd.DataFrame(all_lyrics)
writer = pd.ExcelWriter('all_lyrics.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='lyrics', index=False)
writer.save()
#all_lyrics.to_excel('all_lyrics.xlsx')